# Повний цикл NLP-проекту

## I. Перевірка фактів на достовірність

У межах цієї задачі ви побудуєте систему видобування фактів на правилах, а також інструменти для оцінювання якості роботи цієї системи.

### 1. Домен

Виберіть домен, для якого можна побудувати невелику базу даних на основі [DBPedia](https://dbpedia.org/sparql). Приклади доменів:
- актор і фільми, в яких він знімався;
- письменник і книжки, які він написав;
- музичний гурт і його учасники/концерти/альбоми з роками діяльності/випуску;
- компанія і всі її CEO/CTO з роками діяльності;
- людина і всі її місця роботи з часовими проміжками;
- політик і політичні партії, в яких він брав участь, з роками діяльності;
- винахідник та його винаходи;
- спортсмени і їх команди, матчі, титули тощо.

Проаналізуйте домен і напишіть SPARQL-запит для побудови бази даних.

### 2. Видобування фактів

2.1. Напишіть програму, яка шукає статтю у Вікіпедії про сутність, що належить до вашого домена, та витягає текст цієї статті.

2.2. Напишіть програму, яка опрацьовує текст статті (саме сирий текст, а не таблички, якщо такі є) та витягає з нього інформацію про ваш домен. Цю інформацію ви будете порівнювати зі сформованою базою даних.

### 3. Оцінювання результатів

Розробіть метрику, яка покаже, наскільки інформація, яку ви дістали зі статті, збігається з інформацією в вашій базі даних. Скільки пропущеної інформації? Чи є часткові збіги? (Наприклад, ім'я СЕО збігається лише частково або ім'я СЕО збігається, а роки діяльності різні.)

Додайте ваші спостереження і висновки.

## Приклад

1. Формуємо базу даних про фільми, що знімав Вуді Аллен.

Пишемо SPARQL-запит, щоб отримати всі фільми Вуді Аллена та роки їх випуску:
```
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX res:  <http://dbpedia.org/resource/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT DISTINCT ?movie ?year
WHERE {
	?uri dbo:director res:Woody_Allen .
        OPTIONAL {?uri dct:subject ?cat . 
                  ?cat rdfs:label ?year . 
                  FILTER (regex (?year, '\\d+ films', 'i'))} .
	?uri rdfs:label ?movie .
        FILTER (lang(?movie) = 'en')
}
```

Результат:
```
[("Take the Money and Run", "1969 films")
 ("Annie Hall", "1977 films")
 ("Zelig", "1983 films")
 ("The Purple Rose of Cairo", "1985 films")
 ("Alice (1990 film)", "1990 films")
 ("Hollywood Ending", "2002 films")
 ("Match Point", "2005 films")
 ("Everything You Always Wanted to Know About Sex* (*But Were Afraid to Ask) (film)", "1972 films")
 ("Interiors", "1978 films")
 ("Anything Else", "2003 films")
 ("Melinda and Melinda", "2004 films")
 ("Scoop (2006 film)", "2006 films")
 ("Cassandra's Dream", "2007 films")
 ...]
```

Чистимо результат від зайвої інформації (наприклад, треба видалити з назв фільмів інформацію про рік).

2.1. Скрейпимо інформацію про режисера з Вікіпедії: https://en.wikipedia.org/wiki/Woody_Allen.

2.2. Пишемо набір правил (за допомогою NER, частин мови, синтаксичних дерев, регекспів тощо), які витягають потрібну нам інформацію з тексту:

```
Allen directed, starred in, and co-wrote (with Mickey Rose) Take the Money and Run in 1969, which received positive reviews...

Then came two of Allen's most popular films: Annie Hall and Manhattan. Annie Hall (1977) won four Academy Awards, including...
```

3. Розробляємо метрику, оцінюємо результат і намагаємось покращити якість. Пишемо спостереження і висновки.

In [4]:
# Run a simple query from Python to list all post-rock bands
# Used an out-of-the-box SPARQL wrapper
# Exported in JSON

from SPARQLWrapper import SPARQLWrapper, JSON
import difflib
import requests
import spacy

import mwparserfromhell as parser

from pprint import pprint as pp

def urlify_name(name):
    """Format name to create DBPedia or Wikipedia URL"""
    return name.title().replace(" ", "_")


def build_authors_query():
    """Build query to DBPedia to fetch all known authors.
    Warning: can be slow."""
    
    query = """
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT ?uri ?string
    WHERE {
        ?uri a dbo:Writer .
        OPTIONAL { ?uri rdfs:label ?string . FILTER (lang(?string) = 'en') }
    }
    """
    
    return query


def build_books_query(author_name):
    """Build query to DBPedia to fetch all books by `author_name`.
    Warning: can be slow."""
    
    author_name = urlify_name(author_name)
    
    query = """
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX res:  <http://dbpedia.org/resource/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT DISTINCT ?uri ?string 
    WHERE {
        ?uri rdf:type dbo:Book .
            ?uri dbo:author res:%s .
        OPTIONAL { ?uri rdfs:label ?string . FILTER (lang(?string) = 'en') }
    }
    """ % author_name
    
    return query

def run_query(query):
    """Execute a SPARQL query and return its result"""
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setReturnFormat(JSON)

    sparql.setQuery(query)

    return sparql.query().convert()


def get_all_authors():
    """Get list of all known authors on DBPedia for validation"""
    query = build_authors_query()
    r = run_query(query)
    authors = []
    
    for author in r['results']['bindings']:
        try:
            authors.append(author['string']['value'])
        except KeyError:
            pass
    
    # Return all unique author names
    return set(authors)
    
    
def validate_author(author, authors):
    """Validate author name against a known list of authors"""
    # Exact match
    if author in authors:
        return author
    # If not, attempt to fuzzy match
    else:
        try:
            names = difflib.get_close_matches(author, authors)
            print(names)
            return names[0]
        # If nothing even remotely close is found, return error
        except IndexError:
            return -1

        
def get_books_by_author(author_name):
    """Obtain list of all books by author from DBPedia"""
    
    books = []
    r = run_query(build_books_query(author_name))
    
    for item in r['results']['bindings']:
        books.append((item['string']['value']))
        
    return books


def get_article_text(title):
    """Fetch and return plain text of Wikipedia page about a given author"""
    
    response = requests.get(
        'https://en.wikipedia.org/w/api.php',
        params={
                'action': 'query',
                'format': 'json',
                'titles': title,
                'prop': 'revisions',
                'rvprop': 'content',
            }
        ).json()

    page = next(iter(response['query']['pages'].values()))
    wikicode = page['revisions'][0]['*']
    parsed_wikicode = parser.parse(wikicode)
    text = parsed_wikicode.strip_code()
    
    return text


def para_break(article_text):
    """Break text into paragraphs"""

    nlp = spacy.load("en_core_web_sm")
    sentences = []
    
    try:
        article_text = article_text.strip()
        doc = nlp(article_text)
        sentences.extend([sent.string.strip() for sent in doc.sents])
    except:
        pass
    
    return sentences


def correlate(author_name):
    """Correlate author's list of works with text of Wikipedia page about author"""
    
    # Init final report
    true_facts = 0
    false_facts = 0
    report = ''
    
    # Init list of authors once
    # try:
        # global authors
    # except UnboundLocalError:
    authors = get_all_authors()
    
    # Validate author's name
    author_name = validate_author(author_name, authors)
    
    # Get all books by author
    books = get_books_by_author(author_name)
    
    # Fetch & process wikipedia article
    article_text = get_article_text(author_name)
    paragraphs = para_break(article_text)
    
    # Parse wikipedia sentence by sentence
    eligible = set()
    
    # Select all sentences containing author's name and one of his books
    for paragraph in paragraphs:
        for book in books:
            if book in paragraph and author_name.split()[-1] in paragraph:
                eligible.add(paragraph)
                
    return eligible
    
    
        

author = 'JRR TOlkien'
paragraphs = correlate(author)
pp(paragraphs)
pp(get_books_by_author(author))



['J. R. R. Tolkien']
{"After Tolkien's death, his son Christopher published a series of works based "
 "on his father's extensive notes and unpublished manuscripts, including The "
 'Silmarillion.',
 'Before his death Tolkien negotiated the sale of the manuscripts, drafts, '
 'proofs and other materials related to his then-published works—including The '
 'Lord of the Rings, The Hobbit and Farmer Giles of Ham—to the Department of '
 "Special Collections and University Archives at Marquette University's John "
 'P. Raynor, S.J., Library in Milwaukee, Wisconsin.',
 "Beowulf is one of the most significant influences upon Tolkien's later "
 'fiction, with major details of both The Hobbit and The Lord of the Rings '
 'being adapted from the poem.',
 'During his time at Pembroke College Tolkien wrote The Hobbit and the first '
 'two volumes of The Lord of the Rings, while living at 20 Northmoor Road in '
 'North Oxford (where a blue plaque was placed in 2002).',
 'In 1980 Christopher Tolkien

## Спостереження

1. DBPedia чутлива до точності запиту. Наприклад, шукаючи Дж. Р. Р. Толкіна, за запитом "JRR Tolkien" ми отримуємо порожню відповідь, а коректну відповіь отримаємо лише на "J. R. R. Tolkien". Тому ми вирішили реалізувати **простий валідатор** відомих імен письменників. Хоча такий валідатор здебільшого впорується зі своєю функцією, він, з іншого боку, може неправильно розпізнавати або зовсім ігнорувати певних авторів, наприклад валідатор "впритул не помічає" Вільяма Шекспіра. Це може бути пов’язано з тим, що Шекспір зазначений у DBPedia як dbo:Poet, а не dbo:Writer. Цю проблему можливо потенційно пом’якшити, якщо розширити список "відомих" авторів за рахунок WikiData, парсингу Вікіпедії, Goodreads тощо.
2. Помічаємо досить високу варіативність формулювань у статтях (див. вивід програми вище), розбиття імені автора і назви його твору між реченнями (наприклад: "Tolstoy is considered one of the giants of Russian literature; his works include the novels War and Peace and Anna Karenina and novellas such as Hadji Murad and The Death of Ivan Ilyich.") Тому на нашу думку, **структура статей занадто складна для ручного виділення певних окремих правил, які могли б покрити бодай більшість фактів**. Тому поки що ми обмежилися виводом тих абзаців, де міститься ім’я автора і назва хоча б одного з його творів. 
3. Також спостерігаємо, що у статтях письменників часто згадують за їх прізвищами. Тому щоб краще розпізнавати авторів у статтях, вирішено використовувати не повне ім’я, а прізвище автора. Це дає набагато більшу кількість абзаців, які видає програма. Програму можливо надалі покращити за рахунок пошуку не лише за прізвищем, але і **за псевдонімами** та іншими відомими іменами автора.

In [100]:
nlp = spacy.load("en_core_web_md")
doc = nlp(""""John wrote his book 'The Halberd' in 1892.""")

for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)

John John nsubj wrote
his book book dobj wrote
The Halberd Halberd appos book


## II. Курсовий проект

Для свого курсового проекту визначте остаточні метрики і напишіть програму, яка їх реалізує. Покажіть приклад роботи програми на іграшкових даних (до 10 прикладів реальних чи штучних даних).

### Оцінювання

- 80% - перевірка фактів
- 20% - курсовий проект

## Крайній термін

30.03.2019